In [27]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import random

In [28]:
# classify an image with the given model path
def classify_image(image_path, model, device):
    class_names = [
        'calling',
        'clapping',
        'cycling',
        'dancing',
        'drinking',
        'eating',
        'fightning',
        'hugging',
        'laughing',
        'listening_to_music',
        'running',
        'sitting',
        'sleeping',
        'texting',
        'using_laptop'
    ]

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5729, 0.5379, 0.5069), (0.3056, 0.3022, 0.3096))
    ])
    
    image = Image.open(f'.\data2\test/{image_path}').convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

In [ ]:
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model_path = "models/resnet/resnet_model.pth"  # Replace with your actual model path
model = models.resnet152()
model.fc = nn.Linear(model.fc.in_features, 15)
model.load_state_dict(torch.load("models/resnet/resnet_model.pth"
                                 ,map_location=torch.device("cpu")
))
model.to(device)


# Read the CSV file
# Using absolute path similar to the model path
test_data = pd.read_csv("data2/test/test_labels.csv")
print(f"Test dataset contains {len(test_data)} images")

# Assuming first column is filename and third is the true label
# Adjust these if your CSV has different structure
filename_col = 0
folder_col = 1
label_col = 2

# Process images and compute accuracy
correct = 0
total = 0

for index, row in test_data.iterrows():
    filename = row.iloc[filename_col]
    foldername = row.iloc[folder_col]
    filename = f'{foldername}/{filename}'
    true_label = row.iloc[label_col]
    
    try:
        predicted_label = classify_image(filename, model, device)
        #print(f'Image: {filename}: ({predicted_label}, {true_label})')
        
        if predicted_label == true_label:
            correct += 1
        
        total += 1
        
        # Optional progress update
        if index % 50 == 0:
            print(f"Processed {index}/{len(test_data)} images")
    
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Calculate accuracy
accuracy = 100 * correct / total
print(f"\nAccuracy: {accuracy:.2f}% ({correct}/{total})")

Test dataset contains 1890 images
Error processing calling/Image_10899.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10899.jpg'
Error processing calling/Image_10910.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10910.jpg'
Error processing calling/Image_10919.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10919.jpg'
Error processing calling/Image_10925.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10925.jpg'
Error processing calling/Image_10934.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10934.jpg'
Error processing calling/Image_10936.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10936.jpg'
Error processing calling/Image_10986.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10986.jpg'
Error processing calling/Image_10994.jpg: [Errno 22] Invalid argument: '.\\data2\test/calling/Image_10994.jpg'
Error processing calling/Image_10999.jpg: [Errno 22] Invalid argument: '.\\dat

ZeroDivisionError: division by zero